# Week 7: Clustering II

## Learning Objectives

- Motivate utility of clustering
- Compare PCA & UMAP dimensionality reduction
- Compare k-means & Louvain clustering
- Describe modularity (group connectedness metric)
- Distinguish between an iteration and a pass of Louvain algorithm

### Introduction

Recall from last week, we introduced dimensionality reduction using Principal Component Analysis (PCA) and Uniform Manifold Approximation and Projection (UMAP). These techniques simplify complex datasets by transforming the data into fewer dimensions.

This week, we will build on that foundation by expanding our analytical toolkit. We will explore the strengths and weaknesses of each dimensionality reduction technique, as well as how they can be combined for even better results. Additionally, we will introduce clustering techniques such as k-means and Louvain clustering, which help identify distinct cell populations in complex datasets.

To apply these methods, we will work with single-cell RNA sequencing (scRNA-seq) data. By integrating dimensionality reduction with clustering, we aim to streamline gene expression analysis and better distinguish biologically meaningful cell types and states.

In [ ]:
!pip -q install scanpy
!pip -q install scikit-network
!pip -q install louvain

In [ ]:
# Loading and Preprocessing Dataset
import scanpy as sc

# Plotting
import matplotlib.pyplot as plt

# Dimension Reduction
from sklearn.decomposition import PCA
from umap import UMAP

# Clustering
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import NearestNeighbors
from sknetwork.clustering import Louvain

In [ ]:
# This is a helper function
# Plot multiple clustering results dynamically
def plot_clustering_results(algo_name, param_name, results, umap_embedding, figsize=(12, 8)):
    n_plots = len(results)
    n_rows = (n_plots + 1) // 2 
    fig, axes = plt.subplots(n_rows, 2, figsize=figsize)
    axes = axes.ravel()
    for i, (param_value, clusters) in enumerate(results):
        scatter = axes[i].scatter(umap_embedding[:, 0], umap_embedding[:, 1], c=clusters)
        axes[i].set_title(f'{algo_name} ({param_name}={param_value}, num_clusters={len(set(clusters))})')
        axes[i].set_xlabel('UMAP1')
        axes[i].set_ylabel('UMAP2')
        plt.colorbar(scatter, ax=axes[i])
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    plt.tight_layout()
    plt.show()

### Analyzing Single-Cell Data

A typical scRNA-seq dataset includes:

- Thousands of cells (observations).
- Gene expression levels across thousands of genes (features).

This results in a high-dimensional matrix, where each cell's gene expression profile is a point in this vast space.

**PBMC3K Dataset**

In this module, we will be working with the `pbmc3k` dataset, which is a scRNA-seq dataset made publicly available by 10x Genomics. It consists of roughly 3,000 Peripheral Blood Mononuclear Cells (PBMCs) from a healthy donor.

PBMCs are a diverse mixture of blood cells, including lymphocytes (T cells, B cells, NK cells), monocytes, and dendritic cells, each playing a crucial role in the immune system. This dataset is often used as a benchmark in scRNA-seq data analysis because it encompasses a variety of cell types, making it ideal for exploring dimensionality reduction techniques and clustering algorithms.

(The data is freely available from this [webpage](https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/pbmc3k).)

In the cell below, we will be retrieving the scRNA-seq data from `scanpy`, a toolkit commonly used for analyzing single-cell gene expression data.

In [ ]:
# Load the raw dataset
adata = sc.datasets.pbmc3k()
print(adata.X.shape)

Our raw dataset contains 2,700 PBMCs, with each cell measuring the expression levels of 32,738 unique transcripts. However, analyzing such a large number of genes can be challenging, as many may not provide meaningful insights for distinguishing between cell types. For instance, some genes might not be expressed at all in these cells, while others may show very similar expression patterns.

In addition to providing datasets, `scanpy` provides a variety of tools for processing scRNA-seq data as well. In Week 5, this dataset was preprocessed manually, and with `scapy`, we can preprocess this data with a predefined protocol that comes from  Zheng et al. [2017](https://doi.org/10.1038/ncomms14049). During this preprocessing, each gene's expression is normalized to have a mean of 0 and a standard deviation of 1. After normalization, only the top 1,000 most variable genes are retained, while the rest are filtered out.

In [ ]:
# Preprocess the dataset, keeping only the 1000 most variable genes
sc.pp.recipe_zheng17(adata, n_top_genes=1000, plot=False, copy=False)
print(adata.X.shape)

Now that we’ve preprocessed our data, we face a new challenge: 

- ***How do we interpret and visualize patterns in such a high-dimensional space?***

While scRNA-seq datasets contain biologically meaningful structure, these signals are often obscured by noise, technical variability, and the distances between different clusters lose meaning in thousands of dimensions.

### Extending Dimensionality Reduction

This is where dimensionality reduction becomes important. By condensing the 1,000 features into a smaller set of informative dimensions, we can transform the data into a human-interpretable space while preserving key biological relationships. 

As we saw last week, PCA uses linear trends to reduce data dimensionality by maintaining as much variance as possible. This allows us to reduce noise in high-dimensional data and create 2D plots we can actually understand.

We'll also explore UMAP, a more advanced dimensionality reduction method. Unlike PCA, UMAP can preserve complex, non-linear structures by converting the data into a graph representation. This is especially useful for biological data where relationships aren't always straightforward.

Let's compare what dimensionality reduction looks like for these two methods.

In [ ]:
# Fit PCA from 1,000 dimensions to 2
pca = PCA(n_components=2)
pc_2 = pca.fit_transform(adata.X)

# Visualize the results
plt.scatter(pc_2[:, 0], pc_2[:, 1])
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA of pbmc3k Dataset')
plt.show()

Notice how the PCA reduction appears to compress most data points into a dense central region with limited separation. This compression effect often occurs because PCA focuses on capturing maximum variance along orthogonal components, which can struggle with complex, non-linear relationships in biological data. The first two principal components likely capture only a small percentage of the total variance in our 1,000-dimensional space.

Before applying UMAP, this is a reminder from Week 6 of what the key parameters in UMAP are influencing:
- `n_neighbors`: This controls how UMAP balances local versus global structure. A smaller value will focus on very local structure, while a larger value will push UMAP to look at a broader 'neighborhood.'
- `min_dist`: This sets the minimum distance between embedded points. Lower values create tighter, more compact clusters, while higher values allow points to spread out more.

In [ ]:
# Fit UMAP from 1,000 dimensions to 2
umap = UMAP(n_components=2, n_neighbors=30, min_dist=0.5)
umap_2 = umap.fit_transform(adata.X)

# Visualize the results
plt.scatter(umap_2[:, 0], umap_2[:, 1])
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.title('UMAP of pbmc3k Dataset')
plt.show()

Interestingly, the direct UMAP reduction spreads points more uniformly across the visualization space. While this might initially seem better than PCA, this overly dispersed pattern could indicate that UMAP is struggling to find meaningful local structure when starting from such high dimensionality. The algorithm might be emphasizing small differences that are actually noise rather than a biological signal.

In these two cases, we can see that PCA condenses our information too much, and UMAP doesn't space out our data enough to create observable clusters. This suggests that each method alone might have limitations when applied to our complex biological dataset. 

Let's try a combined approach to see if we can overcome these issues.

In [ ]:
# Fit PCA from 1,000 dimensions to 50
pca = PCA(n_components=50)
pc_50 = pca.fit_transform(adata.X)

# Fit UMAP from 50 dimensions to 2
umap = UMAP(n_components=2, n_neighbors=30, min_dist=0.5)
pc_50_umap_2 = umap.fit_transform(pc_50)

# Visualize the results
plt.scatter(pc_50_umap_2[:, 0], pc_50_umap_2[:, 1])
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.title('Combined PCA & UMAP of pbmc3k Dataset')
plt.show();

This combined approach reveals more distinct clustering patterns. By first using PCA to reduce noise and capture major variance components, then applying UMAP to find non-linear relationships within that cleaner space, we can observe what appear to be biologically meaningful groups. For the exact same reasons that PCA created a better input for our UMAP visualization, it also provides a much better input for our clustering algorithms. By running our clustering on the first 50 principal components (`pc_50`) instead of the original 1,000 genes, we gain key advantages:
- The most important biological signals in our data are exemplified since PCA filtered out significant amounts of noise.
- Distance calculations used by clustering algorithms are more stable and meaningful in a 50-dimensional space than in a 1000-dimensional one.

Therefore, for the remainder of this notebook, we will perform clustering directly on the `pc_50` data we just generated. Our UMAP plot (`pc_50_umap_2`) will serve as the canvas to visualize the results of this clustering. While our visualization now reveals potential patterns in the data, one of the main goals of scRNA-seq data analysis is identifying distinct groups of cells that share similar expression patterns. To do this, we still need a systematic way to identify and define distinct clusters that are present in our dataset. 

### Clustering Algorithms

Clustering is an unsupervised machine learning method that classifies data points based on how close together they are in space. In scRNA seq data, each cell is represented as a point in a high-dimensional space, where each dimension corresponds to the expression level of a gene. Unlike bulk RNA seq data, where expression patterns are more pronounced, variations in scRNA seq are more subtle between individual cell types. Through the use of clustering algorithms, we can identify groups of cells that lie close to one another in this high-dimensional space, revealing shared expression patterns that often reflect distinct cell types, states, or functional subsets. 

To understand how these hidden structures are uncovered, we explore two widely used approaches:
1. K-Means clustering
2. Louvain clustering

#### **K-Means Clustering**

K-means is a centroid-based clustering algorithm that partitions data into k distinct groups by iteratively minimizing variance within a cluster. A centroid is meant to represent the center of a shape, in this case, the center of our data cluster. The algorithm operates in four simple steps:

1. Initialize k centroids randomly (where k is a hyperparameter)
2. Assign each data point to the nearest centroid, forming temporary clusters
3. Update centroids as the mean of all points in each cluster
4. Repeat steps 2 and 3 until the maximum number of iterations is reached or the clusters converge

We can see these steps in action, as well as a plot of the variance with each iteration, in the animation below.
<div align="center">
    <img src="kmeans.gif" height="450" alt="Visualization of k-means algorithm with variance plot">
    <br>
    <a href="https://dashee87.github.io/data%20science/general/Clustering-with-Scikit-with-GIFs/">Source</a>
</div>

K-means is computationally efficient but makes some key assumptions. 
- Clusters are spherical
- Clusters are similarly sized
- Clusters are sensitive to the initial centroid placement

As a result, the choice of k (`n_clusters`) and the random initialization of centroids will affect the outcome of our clustering.

In our clustering examples, we will be using the 50-dimensional PCA reduction.

In [ ]:
# k-means cluster in 50 dimensions 
kmeans = KMeans(n_clusters=4, random_state=0, n_init=10)
kmeans_clusters = kmeans.fit_predict(pc_50)

# Visualize the k-means clusters
plt.scatter(pc_50_umap_2[:, 0], pc_50_umap_2[:, 1], c=kmeans_clusters)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.title('K-means Clusters for pbmc3k Dataset')
plt.colorbar()
plt.show();

print(f"Number of k-means clusters: {len(set(kmeans_clusters))}")

While k-means provides a computationally efficient way to cluster cells, its reliance on spherical cluster assumptions and sensitivity to initialization can limit its effectiveness in scRNA-seq data, where cell populations often form irregular shapes or varying densities. To mitigate this sensitivity to initialization, we have set the `n_init` parameter to 10. This instructs the algorithm to run ten separate times, each with a different random starting position for the cluster centroids, and then select the best result. This approach increases the likelihood of finding a stable and optimal clustering solution.

In the UMAP visualization above, even with this improvement, clusters may overlap or merge if their boundaries are non-linear, and the choice of `n_clusters=4` might not reflect the true biological diversity in the data.

**Q*1. Try varying `n_clusters` in the k-means algorithm in the code block below. How does changing this value impact the clustering results and UMAP visualization?**

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
# WRITE YOUR CODE HERE
# Define up to 4 n_cluster values to test
num_clusters = []    

# Do not modify the code under this comment
results = []
for k in num_clusters:
    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans_clusters = kmeans.fit_predict(pc_50)
    results.append((k, kmeans_clusters))

plot_clustering_results("K-means", "k", results, pc_50_umap_2)

<span style="background-color: #FFD700">**Write your answer below**</span>

Answer here:

---

**Q*2. Since k-means is sensitive to initial centroid placement, try changing the `random_state` and setting `n_init=1`, limiting the number of samples we run, to see how it affects clustering results in the code block below. Are there cases where the algorithm converges to an obviously better/worse arrangement of clusters? Please explain your observations.**

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
# WRITE YOUR CODE HERE
# Define up to 4 random_state values to test
random_states = []    

# Do not modify the code under this comment
results = []
for random_state in random_states:
    kmeans = KMeans(n_clusters=4, random_state=random_state, n_init=1)
    kmeans_clusters = kmeans.fit_predict(pc_50)
    results.append((4, kmeans_clusters))

plot_clustering_results("K-means", "k", results, pc_50_umap_2)

<span style="background-color: #FFD700">**Write your answer below**</span>

Answer here:

---

Other clustering methods like the Louvain algorithm address these challenges by leveraging the connected structure of the data. Instead of assuming spherical clusters, Louvain assumes the data shares some level of connectivity between cell groups, making it better suited for complex/high-dimensional datasets like our scRNA seq data.

#### **Louvain Clustering**

Louvain clustering is a community detection algorithm that optimizes modularity, a metric that measures how well a network is divided into groups. A high modularity score indicates that there are dense connections within communities and sparse connections between different communities. The algorithm iteratively moves nodes between communities to find the move that results in the largest increase in modularity. In this way, it is conceptually similar to a greedy optimization algorithm like gradient descent, which iteratively takes steps in the direction of the steepest ascent.

The algorithm operates in three main steps:

- Construct a graph where each data point is a node with edges weighted by similarity in gene expression. This is typically done using k-nearest neighbors.
- Initialize each node as its own community.
1. Compute the change in modularity for each node and the nodes that are immediately neighbouring it. The nodes connected with the highest change in modularity will be grouped into a new supercommunity.
2. Aggregate each community into nodes that represent each one. These super nodes maintain the representation of connectedness between communities with weighted connections. Links between super nodes are weighted to represent the number of connections between those communities before aggregation. Similarly, super nodes contain self-loops to represent the connections within a single community before aggregation.
3. Repeat steps 1 and 2 until the maximum number of iterations is reached or there are no further increases in modularity. With each repeat, the graph becomes smaller and smaller.

<div align="center">
    <img src="louvain.png" width="1000" alt="Visualization of Louvain algorithm with graphs">
    <br>
    <a href="https://medium.com/data-science/louvain-algorithm-93fde589f58c">Source</a>
</div>

The Louvain algorithm is one of the most widely used clustering methods in scRNA-seq data analysis. Unlike k-means, Louvain can cluster irregular shapes and automatically infers the number of clusters. Furthermore, the Louvain algorithm focuses on local relationships instead of global centroids, making it more robust to noise and influences from initial conditions. Together, these features make Louvain a scalable and reliable choice for unraveling the communities within scRNA-seq data.

In [ ]:
# Create graph using k-nearest neighbours in 50 dimensions
knn = NearestNeighbors(n_neighbors=24)
knn_result = knn.fit(pc_50)
adjacency_matrix = knn_result.kneighbors_graph(pc_50).toarray()

# Louvain cluster on graph
louvain = Louvain(resolution=1, modularity='dugue', random_state=0)
louvain_clusters = louvain.fit_predict(adjacency_matrix)

# Visualize the Louvain clusters
plt.scatter(pc_50_umap_2[:, 0], pc_50_umap_2[:, 1], c=louvain_clusters)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.title('Louvain Clusters for pbmc3k Dataset')
plt.colorbar()
plt.show();

print(f"Number of Louvain clusters: {len(set(louvain_clusters))}")

Here, we can see that the Louvain algorithm is able to capture clusters that are distinct from the k-means clusters we saw earlier. The use of the graph structure allows for irregular shapes to be better clustered together. This aligns well with the often complex, non-spherical structures present in high-dimensional biological data like scRNA-seq.

However, the exact clustering result from Louvain is influenced by hyperparameters. One of the most critical hyperparameters is `resolution`. This hyperparameter directly affects the granularity of the clustering. Lower resolution encourages merging communities, resulting in fewer, larger clusters. Higher resolution penalizes cluster merging, leading to more, smaller clusters. By adjusting the resolution, we may be able to distinguish between different levels of lineage. For example, we might only be able to differentiate between broad cell lineages (e.g. lymphocytes, myeloid cells) at low resolutions, but with higher resolutions we may be able to tell different types of T-cells apart and distinguish between resting vs activated cells.

---
**Q*3. Try varying `resolution` in the Louvain algorithm in the code block below between 0 and 3. How does changing this value impact the clustering results and UMAP visualization?**

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
# WRITE YOUR CODE HERE
# Define up to 4 resolution values to test
resolutions = []    

# Do not modify the code under this comment
results = []
for resolution in resolutions:
    louvain = Louvain(resolution=resolution, modularity='dugue', random_state=0)
    louvain_clusters = louvain.fit_predict(adjacency_matrix)
    results.append((resolution, louvain_clusters))

plot_clustering_results("Louvain", "resolution", results, pc_50_umap_2)

<span style="background-color: #FFD700">**Write your answer below**</span>

Answer here:

---

**Q*4. Try changing the `random_state` to see how it affects clustering results in the code block below. How do these results compare to k-means under different `random_states`? Are these results more or less sensitive to initial conditions? Please explain your observations.**

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
# WRITE YOUR CODE HERE
# Define up to 4 random_state values to test
random_states = []

# Do not modify the code under this comment
results = []
for random_state in random_states:
    louvain = Louvain(resolution=1.0, modularity='dugue', random_state=random_state)
    louvain_clusters = louvain.fit_predict(adjacency_matrix)
    results.append((1.0, louvain_clusters))

plot_clustering_results("Louvain", "resolution", results, pc_50_umap_2)

<span style="background-color: #FFD700">**Write your answer below**</span>

Answer here:

---

### Scanpy Version

With this foundation established, it's useful to see how these concepts translate into the streamlined workflows common in the field. `scanpy` encapsulates these analytical steps very nicely. While we've dissected the process for learning, `scanpy` allows us to apply this knowledge efficiently. Let's look at how our analysis pipeline appears when implemented using `scanpy`'s high-level functions, demonstrating the power of integrating these steps within a dedicated scRNA-seq analysis environment.

As we saw previously in this week's notebook, `scanpy` comes equiped with example datasets for testing these pipelines (in this case we used the PBMC3K dataset) and it contains a variety of preprocessing methods such as `sc.pp.recipe_zheng17(...)` though more in the <a href="https://scanpy.readthedocs.io/en/1.11.x/api/preprocessing.html#recipes">scapy docs</a>.

In [ ]:
# Load the dataset
adata = sc.datasets.pbmc3k()

# Preprocess the dataset, keeping only the 1000 most variable genes
sc.pp.recipe_zheng17(adata, n_top_genes=1000, plot=False, copy=False)

We can use `sc.tl.pca(...)` to perform PCA. It will get stored in the `adata` structure under `adata.obsm['X_pca']`.

In [ ]:
# Run PCA (stores PCs in adata.obsm['X_pca'])
sc.tl.pca(adata, svd_solver='arpack')

For both UMAP and the Louvain algorithm, we require a graph data structure as input. `sc.pp.neighbors(...)` allows us to build a graph of our data using the k-nearest neighbors algorithm.

Note: If you are curious to learn more about graphs, you can check out this <a href="https://www.geeksforgeeks.org/introduction-to-graphs-data-structure-and-algorithm-tutorials/">link</a>, though we don't expect you to understand any of these high-level concepts for this course. We only expect you to know these algorithms use them to perform computations.

In [ ]:
# Compute the neighborhood graph (needed for UMAP and Louvain)
# Uses PCs by default (controlled by use_rep='X_pca')
sc.pp.neighbors(adata, n_neighbors=24, n_pcs=50)

With this graph, we can now call `sc.tl.umap(...)`, reducing the dimension on the PCA projections and store it under `adata.obsm['X_umap']` for us to plot in 2D.

In [ ]:
# Run UMAP (stores UMAP coords in adata.obsm['X_umap'])
sc.tl.umap(adata)

Using the graph representation, we can also call `sc.tl.louvain(...)` to cluster our data using the Louvain algorithm. Other clustering methods can also be found in the <a href="https://scanpy.readthedocs.io/en/1.11.x/api/tools.html#clustering-and-trajectory-inference">scanpy docs</a>.

In [ ]:
# Run Louvain clustering (stores cluster labels in adata.obs['louvain'])
# Resolution parameter is directly available.
sc.tl.louvain(adata, resolution=1.0, random_state=0)

Lastly, we can create a plot of the results from our clustering with the PCA+UMAP dimensionality reduction using `sc.pl.umap(...)`. `plt.show` allows the plot to be visualized in the cell output, and we can count the number of clusters observed from the louvain algorithm using `len(adata.obs['louvain'].cat.categories)`.

Note: Though it may still say UMAP here and in an above code cell, `sc.pl` is for plotting data while the call to `sc.pp` is used for preprocessing data.

In [ ]:
# Visualize the results using scanpy's plotting function
sc.pl.umap(adata, color='louvain', title='Scanpy Louvain Clusters')
plt.show()

print(f"Number of Scanpy Louvain clusters: {len(adata.obs['louvain'].cat.categories)}")

## **Graded Exercise: (6 marks)**

**GQ*1: Based on the exploration using different random_state values in the notebook, how does the stability (consistency of results across runs) of Louvain clustering generally compare to that of k-means clustering for this dataset? (1 mark)**

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here:

---

**GQ*2: We applied PCA to reduce the data to 50 dimensions, followed by UMAP to reduce it further to 2 dimensions for visualization. Explain the primary benefit of each step in this specific two-step process, compared to applying only PCA or only UMAP directly from the initial 1000 gene data. (2 marks)**

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here:

---

**GQ*3: Using the Scanpy code, rerun the Louvain clustering algorithm with 20 nearest neighbors, 30 principal components, and a resolution of 1.5. (3 marks)**

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
# WRITE YOUR CODE HERE

# Load the dataset


# Preprocess the dataset, keeping only the 1000 most variable genes


# Run PCA (stores PCs in adata.obsm['X_pca'])


# Compute the neighborhood graph (needed for UMAP and Louvain)


# Uses PCs by default (controlled by use_rep='X_pca')


# Run UMAP (stores UMAP coords in adata.obsm['X_umap'])


# Run Louvain clustering (stores cluster labels in adata.obs['louvain'])


# Resolution parameter is directly available.


# Visualize the results using Scanpy's plotting function



---
## Conclusion

This week, we explored how clustering algorithms help us identify distinct cell populations within complex scRNA-seq datasets like the pbmc3k. While raw high-dimensional data is difficult to interpret directly, dimensionality reduction techniques are crucial for visualizing potential structures and assessing the quality of our clustering results. By investigating how we can combine PCA and UMAP techniques to reduce noise and maintain structure in our data, we were able to successfully plot meaningful patterns in two dimensions for visualization.

We were also able to look at different clustering algorithms like k-means and Louvain to distinguish between different cell types based on their gene expression patterns. While k-means is fairly straightforward, it assumes clusters are spherical and is very sensitive to the random starting points, sometimes leading to different results across runs. On the other hand, the Louvain algorithm is more robust in the sense that it can handle irregularly shaped clusters and exhibits much more stability when given different random starting points. These methods are not without limitations. We still introduce some bias into our choice of hyperparameters (e.g., `n_clusters`, `resolution`, `modularity`), which may cause our results to slightly deviate from ground truth clusters. Additionally, methods that are dependent on distance metrics (e.g., k-means, k-nearest neighbors) cause some complications when we are working with data in higher dimensions. Distance is less meaningful in higher dimensions, and there are more advanced clustering techniques that aim to address this issue directly by combining dimensionality reduction and clustering more harmoniously. 

Together, these explorations highlight the importance of choosing appropriate clustering methods when working with scRNA-seq data.